In [4]:
import os
import tensorflow as tf

In [24]:
filename = "out.csv"

features = tf.placeholder(tf.int32, shape=[5], name='features')
date = tf.placeholder(tf.string, name='DATE')
total = tf.reduce_sum(features, name='total')

printerop = tf.Print(total, [date, features, total], name='printer')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    with open(filename) as inf:
        # Skip header
        next(inf)
        for line in inf:
            # Read data, using python, into our features
            row, DATE, funds_rate, oil_cpi, high_tax_rate, low_tax_rate, unemployment, real_gdp = line.strip().split(",")
            funds_rate = float(funds_rate)
            oil_cpi = float(oil_cpi)
            try:
                high_tax_rate = float(high_tax_rate)
            except:
                pass
            low_tax_rate = float(low_tax_rate)
            unemployment = float(unemployment)
            real_gdp = float(real_gdp)
            
            # Run the Print ob
            total = sess.run(printerop, feed_dict={features: [funds_rate, oil_cpi, high_tax_rate, low_tax_rate, unemployment], date:DATE})
            print(DATE, total)

('1970-01-01', 113)
('1970-04-01', 28)
('1970-07-01', 29)
('1970-10-01', 28)
('1971-01-01', 110)
('1971-04-01', 28)
('1971-07-01', 29)
('1971-10-01', 28)
('1972-01-01', 110)
('1972-04-01', 27)
('1972-07-01', 27)
('1972-10-01', 28)
('1973-01-01', 112)
('1973-04-01', 31)
('1973-07-01', 34)
('1973-10-01', 36)
('1974-01-01', 127)
('1974-04-01', 46)
('1974-07-01', 51)
('1974-10-01', 51)
('1975-01-01', 133)
('1975-04-01', 48)
('1975-07-01', 50)
('1975-10-01', 51)
('1976-01-01', 133)
('1976-04-01', 49)
('1976-07-01', 50)
('1976-10-01', 51)
('1977-01-01', 137)
('1977-04-01', 54)
('1977-07-01', 55)
('1977-10-01', 57)
('1978-01-01', 141)
('1978-04-01', 57)
('1978-07-01', 58)
('1978-10-01', 60)
('1979-01-01', 147)
